In [1]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import numpy as np
from sklearn.externals import joblib
import sys
sys.path.append('../../../code/utils/')
sys.path.append('../../../code/feature/')
import data_utils as du
import perf_utils as pu
import gc
from scipy.sparse import find

In [2]:
gpu_count = torch.cuda.device_count()
print('GPU count:', gpu_count)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

GPU count: 2
device cuda:0


In [3]:
torch.manual_seed(20180429)    # reproducible
# Hyper Parameters
EPOCH = 1                                       #Haven't figured out, let these just be here
BATCH_SIZE = 256
LR = 5e-5         # learning rate

In [4]:
def csr_to_tensor(csr_matrix, size):
    data = csr_matrix.data
    indices = csr_matrix.indices
    
    # http://pytorch.org/docs/stable/sparse.html
    i = torch.LongTensor([[0, num] for num in indices], device=device)
    v = torch.FloatTensor(data.astype(np.float), device=device)
    result_tensor = torch.sparse.FloatTensor(i.t(), v, size, device=device).to_dense()
    return result_tensor

In [5]:
# Here you should convert data into formats that torch accepts
# Make a dataloader
class MyDataset(Data.Dataset):
    def __init__(self, x, y):
        self.input = x
        self.label = y

    def __getitem__(self, index):#返回的是tensor
        # convert self.input[index] to tensor
        input_item = self.input[index]
        x = csr_to_tensor(input_item, torch.Size([1,self.input.shape[1]]))
        
        # convert self.label[index] to tensor
        y = torch.tensor(self.label[index])
        return x, y

    def __len__(self):
        return len(self.label)

In [6]:
# Here we should probably define the auto-encoder
class AutoEncoder(nn.Module):
    def __init__(self, input_size):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.Tanh(),
            nn.Linear(128, 32),
        ).to(device)
        self.encoder = nn.DataParallel(self.encoder)
        self.decoder = nn.Sequential(
            nn.Linear(32, 128),
            nn.Tanh(),
            nn.Linear(128, input_size),
        ).to(device)
        self.decoder = nn.DataParallel(self.decoder)

    def forward(self, x):
        input = x.to(device)
        encoded = self.encoder(input)
        decoded = self.decoder(encoded)
        return encoded, decoded


In [7]:
# load data
df_train = du.load_raw_data("train")
gc.collect()

# train_X: (8798814, 374251)
# train_y: (8798814,)

train_X = joblib.load('../../../../../zhangez698/TencentAlgo2018/playground/Elvin/autoencoder/xxx.pkl') # on Server
train_y = joblib.load('../../../../../zhangez698/TencentAlgo2018/playground/Elvin/autoencoder/yyy.pkl')
#train_X, train_y = du.get_set(df_train, test = False, features_u_want = ['house', 'interest2', 'kw1', 'kw2',  'appIdInstall'], a_features_u_want = ['aid', 'productId'])

print(train_X[0]) # eg. (0, 1) is the position of non-zero data whose value is 1
del df_train
gc.collect()

  (0, 1)	1
  (0, 7)	1
  (0, 10)	1
  (0, 23)	1
  (0, 31)	1
  (0, 728)	1
  (0, 893)	1
  (0, 908)	1
  (0, 923)	1
  (0, 941)	1
  (0, 985)	1
  (0, 986)	1
  (0, 1004)	1
  (0, 1092)	1
  (0, 1103)	1
  (0, 1114)	1
  (0, 1251)	1
  (0, 28269)	1
  (0, 114175)	1
  (0, 122177)	1
  (0, 177443)	1
  (0, 177584)	1
  (0, 310357)	1
  (0, 322279)	1
  (0, 323861)	1
  (0, 324111)	1
  (0, 324797)	1
  (0, 329213)	1
  (0, 330004)	1
  (0, 342260)	1
  (0, 348133)	1
  (0, 412989)	1
  (0, 419204)	1
  (0, 419206)	1
  (0, 419260)	1
  (0, 419395)	1
  (0, 419567)	1
  (0, 419658)	1
  (0, 419775)	1
  (0, 419790)	1
  (0, 419825)	1
  (0, 419859)	1


7

In [8]:
with pu.profiler("Setting up autoencoder"):
    autoencoder = AutoEncoder(train_X.shape[1])
    autoencoer = nn.DataParallel(autoencoder)
    autoencoder.to(device)
    optimizer = torch.optim.Adam(autoencoder.parameters(), lr=LR, weight_decay = 1e-5)
    criterion = nn.MSELoss()

[15:55:59] Finish Setting up autoencoder. △M: +1.48GB. △T: 4.8 seconds.


In [9]:
train_dataset = MyDataset(train_X, train_y)
train_loader = Data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [10]:
for epoch in range(EPOCH):
    for step, (x, y) in enumerate(train_loader): # step: batch index; x.shape: [BATCH_SIZE, 1, 374251]; y.shape: [BATCH_SIZE]   
        b_x = torch.tensor(x).to(device)                   # batch x
        b_y = torch.tensor(x).to(device)
        b_label = torch.tensor(y).to(device)               # batch label

        encoded, decoded = autoencoder(b_x)

        loss = criterion(decoded, b_y)      # mean square error
        optimizer.zero_grad()               # clear gradients for this training step since gradients are accumulated in the process
        loss.backward()                     # backpropagation, compute gradients
        optimizer.step()                    # apply gradients. "The function can be called once the gradients are computed using eg. backward()"

        # Currently the training speed is really slow, change the numbers below according to what you need
        if step % 10 == 0:
            print('epoch [{}/{}], samples[{}/{}], loss:{:.8f}'
                  .format(epoch + 1, EPOCH, (step + 1) * BATCH_SIZE, train_X.shape[0], loss.item()))
            print('end time: {}'.format(pu.get_time_str()))
            print('-' * 80)
#         if step == 100:
#             break;

epoch [1/1], samples[256/8798814], loss:0.00593919
end time: 15:56:06
--------------------------------------------------------------------------------
epoch [1/1], samples[2816/8798814], loss:0.00531732
end time: 15:56:19
--------------------------------------------------------------------------------
epoch [1/1], samples[5376/8798814], loss:0.00492823
end time: 15:56:31
--------------------------------------------------------------------------------
epoch [1/1], samples[7936/8798814], loss:0.00466196
end time: 15:56:43
--------------------------------------------------------------------------------
epoch [1/1], samples[10496/8798814], loss:0.00445479
end time: 15:56:56
--------------------------------------------------------------------------------
epoch [1/1], samples[13056/8798814], loss:0.00426609
end time: 15:57:08
--------------------------------------------------------------------------------
epoch [1/1], samples[15616/8798814], loss:0.00410123
end time: 15:57:21
---------------

epoch [1/1], samples[138496/8798814], loss:0.00119879
end time: 16:07:19
--------------------------------------------------------------------------------
epoch [1/1], samples[141056/8798814], loss:0.00118655
end time: 16:07:32
--------------------------------------------------------------------------------
epoch [1/1], samples[143616/8798814], loss:0.00116409
end time: 16:07:44
--------------------------------------------------------------------------------
epoch [1/1], samples[146176/8798814], loss:0.00114781
end time: 16:07:57
--------------------------------------------------------------------------------
epoch [1/1], samples[148736/8798814], loss:0.00112310
end time: 16:08:09
--------------------------------------------------------------------------------
epoch [1/1], samples[151296/8798814], loss:0.00110775
end time: 16:08:22
--------------------------------------------------------------------------------
epoch [1/1], samples[153856/8798814], loss:0.00108825
end time: 16:08:34
---

epoch [1/1], samples[276736/8798814], loss:0.00052438
end time: 16:18:37
--------------------------------------------------------------------------------
epoch [1/1], samples[279296/8798814], loss:0.00050361
end time: 16:18:49
--------------------------------------------------------------------------------
epoch [1/1], samples[281856/8798814], loss:0.00049404
end time: 16:19:02
--------------------------------------------------------------------------------
epoch [1/1], samples[284416/8798814], loss:0.00049019
end time: 16:19:14
--------------------------------------------------------------------------------
epoch [1/1], samples[286976/8798814], loss:0.00048515
end time: 16:19:27
--------------------------------------------------------------------------------
epoch [1/1], samples[289536/8798814], loss:0.00046965
end time: 16:19:39
--------------------------------------------------------------------------------
epoch [1/1], samples[292096/8798814], loss:0.00045893
end time: 16:19:52
---

epoch [1/1], samples[414976/8798814], loss:0.00025692
end time: 16:29:57
--------------------------------------------------------------------------------
epoch [1/1], samples[417536/8798814], loss:0.00025403
end time: 16:30:09
--------------------------------------------------------------------------------
epoch [1/1], samples[420096/8798814], loss:0.00024745
end time: 16:30:22
--------------------------------------------------------------------------------
epoch [1/1], samples[422656/8798814], loss:0.00024419
end time: 16:30:34
--------------------------------------------------------------------------------
epoch [1/1], samples[425216/8798814], loss:0.00024380
end time: 16:30:47
--------------------------------------------------------------------------------
epoch [1/1], samples[427776/8798814], loss:0.00025545
end time: 16:30:59
--------------------------------------------------------------------------------
epoch [1/1], samples[430336/8798814], loss:0.00023607
end time: 16:31:12
---

epoch [1/1], samples[553216/8798814], loss:0.00017294
end time: 16:41:18
--------------------------------------------------------------------------------
epoch [1/1], samples[555776/8798814], loss:0.00017630
end time: 16:41:31
--------------------------------------------------------------------------------
epoch [1/1], samples[558336/8798814], loss:0.00017533
end time: 16:41:44
--------------------------------------------------------------------------------
epoch [1/1], samples[560896/8798814], loss:0.00016589
end time: 16:41:56
--------------------------------------------------------------------------------
epoch [1/1], samples[563456/8798814], loss:0.00017846
end time: 16:42:09
--------------------------------------------------------------------------------
epoch [1/1], samples[566016/8798814], loss:0.00017395
end time: 16:42:21
--------------------------------------------------------------------------------
epoch [1/1], samples[568576/8798814], loss:0.00017466
end time: 16:42:34
---

KeyboardInterrupt: 

In [ ]:
# the speed doesn't improve: https://github.com/pytorch/pytorch/issues/3917
# uneven gpu util